In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression



# Load the dataset
file_path = 'Resources/US_Accidents_2022_Phoenix_Metro2.csv'
df = pd.read_csv(file_path, low_memory=False)

# Drop columns with too many missing values
df.drop(columns=['End_Lat', 'End_Lng'], inplace=True)

# Handle missing values in other columns
df.dropna(inplace=True)

# Convert date columns to datetime and then to numerical values (e.g., ordinal)
df['Accident_Date'] = pd.to_datetime(df['Accident_Date']).map(pd.Timestamp.toordinal)

# Create labels and features
y = df['Severity']
X = df.drop(columns=['Severity'])

# Identify non-numeric columns
non_numeric_columns = X.select_dtypes(include=['object']).columns

# One-hot encode categorical columns
X = pd.get_dummies(X, columns=non_numeric_columns, drop_first=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Instantiate and fit the Logistic Regression model
clf = LogisticRegression(random_state=1, max_iter=1000)
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Combine classes into binary categories
df['binary_target'] = df['Severity'].apply(lambda x: 0 if x in [1, 2] else 1)

# Features and new binary target
X = df.drop(columns=['Severity', 'binary_target'])
y = df['binary_target']

# One-hot encode categorical columns again
X = pd.get_dummies(X, columns=non_numeric_columns, drop_first=True)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Confusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)


[[  0   2   0   0]
 [  0 174   0   0]
 [  0  11   0   0]
 [  0   2   0   0]]
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         2
           2       0.92      1.00      0.96       174
           3       0.00      0.00      0.00        11
           4       0.00      0.00      0.00         2

    accuracy                           0.92       189
   macro avg       0.23      0.25      0.24       189
weighted avg       0.85      0.92      0.88       189

Confusion Matrix:
 [[210   1]
 [ 16   0]]

Classification Report:
               precision    recall  f1-score   support

           0       0.93      1.00      0.96       211
           1       0.00      0.00      0.00        16

    accuracy                           0.93       227
   macro avg       0.46      0.50      0.48       227
weighted avg       0.86      0.93      0.89       227



In [10]:
from imblearn.over_sampling import SMOTE

# Load the dataset
file_path = 'Resources/US_Accidents_2022_Phoenix_Metro2.csv'
df = pd.read_csv(file_path, low_memory=False)

# Drop columns with too many missing values
df.drop(columns=['End_Lat', 'End_Lng'], inplace=True)

# Handle missing values in other columns
df.dropna(inplace=True)

# Convert date columns to datetime and then to numerical values (e.g., ordinal)
df['Accident_Date'] = pd.to_datetime(df['Accident_Date']).map(pd.Timestamp.toordinal)

# Combine classes into binary categories
df['binary_target'] = df['Severity'].apply(lambda x: 0 if x in [1, 2] else 1)

# Create labels and features
y = df['binary_target']
X = df.drop(columns=['Severity', 'binary_target'])

# Identify non-numeric columns
non_numeric_columns = X.select_dtypes(include=['object']).columns

# One-hot encode categorical columns
X = pd.get_dummies(X, columns=non_numeric_columns, drop_first=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Apply SMOTE to balance the classes
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Train the logistic regression model with class weights
model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(X_train_smote, y_train_smote)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Confusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)


Confusion Matrix:
 [[210   1]
 [ 16   0]]

Classification Report:
               precision    recall  f1-score   support

           0       0.93      1.00      0.96       211
           1       0.00      0.00      0.00        16

    accuracy                           0.93       227
   macro avg       0.46      0.50      0.48       227
weighted avg       0.86      0.93      0.89       227



In [12]:
# One-hot encode categorical columns
df_encoded = pd.get_dummies(df, columns=non_numeric_columns, drop_first=True)

# Compute correlation between features and target variable
correlation = df_encoded.corr()['binary_target'].abs().sort_values(ascending=False)

# Display correlation values
print(correlation)


binary_target                                                               1.000000
Severity                                                                    0.886905
Description_Lane blocked due to crash on I-17 Southbound at AZ-101-LOOP.    0.185686
Street_N Black Canyon Hwy                                                   0.181479
Street_I-10 W                                                               0.172292
                                                                              ...   
Weather_Condition_Mostly Cloudy                                             0.004044
Humidity(%)                                                                 0.001382
Wind_Direction_VAR                                                          0.001158
Bump                                                                             NaN
Traffic_Calming                                                                  NaN
Name: binary_target, Length: 2337, dtype: float64


In [8]:
# Identify non-numeric columns
non_numeric_columns = X.select_dtypes(include=['object']).columns

# One-hot encode categorical columns
X = pd.get_dummies(X, columns=non_numeric_columns, drop_first=True)


In [14]:
from imblearn.over_sampling import SMOTE

# Load the dataset
file_path = 'Resources/US_Accidents_2022_Phoenix_Metro2.csv'
df = pd.read_csv(file_path, low_memory=False)

# Drop columns with too many missing values
df.drop(columns=['End_Lat', 'End_Lng'], inplace=True)

# Handle missing values in other columns
df.dropna(inplace=True)

# Convert date columns to datetime and then to numerical values (e.g., ordinal)
df['Accident_Date'] = pd.to_datetime(df['Accident_Date']).map(pd.Timestamp.toordinal)

# Combine classes into binary categories
df['binary_target'] = df['Severity'].apply(lambda x: 0 if x in [1, 2] else 1)

# One-hot encode categorical columns
df_encoded = pd.get_dummies(df, drop_first=True)

# Compute correlation between features and target variable
correlation = df_encoded.corr()['binary_target'].abs().sort_values(ascending=False)

# Display correlation values
print(correlation)


binary_target                                                               1.000000
Severity                                                                    0.886905
Description_Lane blocked due to crash on I-17 Southbound at AZ-101-LOOP.    0.185686
Street_N Black Canyon Hwy                                                   0.181479
Street_I-10 W                                                               0.172292
                                                                              ...   
Weather_Condition_Mostly Cloudy                                             0.004044
Humidity(%)                                                                 0.001382
Wind_Direction_VAR                                                          0.001158
Bump                                                                             NaN
Traffic_Calming                                                                  NaN
Name: binary_target, Length: 2337, dtype: float64


In [16]:
from imblearn.over_sampling import SMOTE

# Load the dataset
file_path = 'Resources/US_Accidents_2022_Phoenix_Metro2.csv'
df = pd.read_csv(file_path, low_memory=False)

# Drop columns with too many missing values
df.drop(columns=['End_Lat', 'End_Lng'], inplace=True)

# Handle missing values in other columns
df.dropna(inplace=True)

# Convert date columns to datetime and then to numerical values (e.g., ordinal)
df['Accident_Date'] = pd.to_datetime(df['Accident_Date']).map(pd.Timestamp.toordinal)

# Combine classes into binary categories
df['binary_target'] = df['Severity'].apply(lambda x: 0 if x in [1, 2] else 1)

# One-hot encode categorical columns
df_encoded = pd.get_dummies(df, drop_first=True)

# Print the first few rows of the encoded DataFrame
print(df_encoded.head())

# Compute correlation between features and target variable
correlation = df_encoded.corr()['binary_target'].abs().sort_values(ascending=False)

# Display correlation values
print(correlation)


   Accident_Date  Severity  Start_Lat   Start_Lng  Distance(mi)  Zipcode  \
1         738165         2  33.494709 -112.056328         0.000    85014   
2         738269         2  33.406713 -112.061288         0.181    85040   
3         738158         2  33.497898 -112.186309         0.053    85033   
4         738251         2  33.552662 -112.209618         0.000    85303   
5         738412         2  33.428950 -112.099768         6.607    85007   

   Temperature(F)  Wind_Chill(F)  Humidity(%)  Pressure(in)  ...  \
1            58.0           58.0         27.0         29.14  ...   
2            75.0           75.0          9.0         28.72  ...   
3            62.0           62.0         22.0         28.95  ...   
4            66.0           66.0         36.0         28.73  ...   
5            95.0           95.0         21.0         28.61  ...   

   Weather_Condition_Light Drizzle / Windy  Weather_Condition_Light Rain  \
1                                    False                

In [18]:
# Compute correlation between features and target variable
correlation = df_encoded.corr()['binary_target'].abs().sort_values(ascending=False)

# Display correlation values
print(correlation)


binary_target                                                               1.000000
Severity                                                                    0.886905
Description_Lane blocked due to crash on I-17 Southbound at AZ-101-LOOP.    0.185686
Street_N Black Canyon Hwy                                                   0.181479
Street_I-10 W                                                               0.172292
                                                                              ...   
Weather_Condition_Mostly Cloudy                                             0.004044
Humidity(%)                                                                 0.001382
Wind_Direction_VAR                                                          0.001158
Bump                                                                             NaN
Traffic_Calming                                                                  NaN
Name: binary_target, Length: 2337, dtype: float64


In [20]:
print("Before SMOTE:")
print(y_train.value_counts())
print("After SMOTE:")
print(pd.Series(y_train_smote).value_counts())


Before SMOTE:
binary_target
0    490
1     38
Name: count, dtype: int64
After SMOTE:
binary_target
0    490
1    490
Name: count, dtype: int64
